# __Bond Spread Alert Project__
This project will do the following:
* Develop my ability to work with APIs by drawing information from the Federal Reserve Economic Data (FRED) database
* Pull bond rate data at regular intervals from FRED
* Calculate bond spreads (particularly the 10-year:3-month spread or 10-year:2-month spread)
* Include a graph of the rates (or spread)
* Alert the user if the spread is negative (indicating an inversion in the yield curve) for the user-designated time period

### __Reasoning for the project__
The past seven recessions have been preceeded by an inversion of the (bond) yield curve.  The yield curve is considered a leading indicator (with a time frame of roughly 12-18 months) when predicting recessions.

The 10-year:3-month spread has been indicated by the Fed (link here) as the most reliable predictor of a recession, while Goldman Sachs asserts that the 10-year:2-year spread is more reliable (documented [here](https://www.cnbc.com/2019/03/26/goldman-says-investors-are-looking-at-the-wrong-bond-market-indicator-and-recession-is-less-likely.html)).  Professor Campbell Harvey (the one who discovered the correlation) uses a [5-year:90-day spread](https://www.fuqua.duke.edu/duke-fuqua-insights/harvey-yield-curve). He says, "Importantly, there is only a prediction of recession when the T-bill yield is greater than the five-year yield for a full quarter."

### __Future Development Ideas__
In future projects, the following should be considered:
- CBOE Volatility Index - estimating the 30-day expected volatitlity of the S&P 500
- Credit spreads

### __Data Sources__
This program will utilize the categories lookup page of the FRED dataset ([here](https://fred.stlouisfed.org/categories)).  The Series ID can then be added - found, for example, [here](https://fred.stlouisfed.org/series/T10Y2Y) - to the URL variable above (after where it says 'series?series_id=').

For this project, I've chosen to utilize the following categories (series IDs in parenthese following):
- 10-Year Treasury Constant Maturity Minus 2-Year Treasury Constant Maturity (T10Y2Y)
- 10-Year Treasury Constant Maturity Minus 3-Month Treasury Constant Maturity (T10Y3M)

In [1]:
#import the native json library
import json
#import the HTTP library
import requests
import pandas as pd

import numpy as np

#API Key (key variable), obtained through the FRED (St. Louis) website (url variable)
key = '53f359cfaa4a9361acba52d85e9f3dd6'
url_2year = 'https://api.stlouisfed.org/fred/series/observations?series_id=T10Y2Y&api_key={}&file_type=json'.format(key)
url_3month = 'https://api.stlouisfed.org/fred/series/observations?&series_id=T10Y3M&api_key={}&file_type=json'.format(key)
parameters = {"sort_order":        'desc',       #descending sort order
              "observation_start": '2008-01-01', #earliest start date (YYYY-MM-DD)
              "frequency":         'd',          #daily
              #"output_type":      '4'           #Observations, initial release only - pulls the current date, use for updates
             }

#Sends the GET request to the FRED server
response_2year = requests.get(url_2year, params = parameters)
response_3month = requests.get(url_3month, params = parameters)

#Prints the status code of the request returned by the FRED server
print("The following information pertains to the 10-year:2-year spread data")
print(response_2year.status_code, '\n')

#Request metadata is stored in the response headers, accessible with the headers property of a response object
print(response_2year.headers, '\n')

#The 'content-type' key tells us the type of information we're working with (and consequently, how to decode it)
print(response_2year.headers['content-type'], '\n', '\n') 

#The same (above) information for the 10-year:3-month spread
print("The following information pertains to the 10-year:3-month spread data")
print(response_3month.status_code, '\n')
print(response_3month.headers, '\n')
print(response_3month.headers['content-type']) 

The following information pertains to the 10-year:2-year spread data
200 

{'Content-Type': 'application/json; charset=UTF-8', 'Content-Length': '14547', 'Server': 'Apache', 'Last-Modified': 'Thu, 13 Jun 2019 20:48:01 GMT', 'Expires': 'Thu, 13 Jun 2019 22:43:31 GMT', 'x-rate-limit-limit': '120', 'x-rate-limit-remaining': '120', 'Vary': 'Accept-Encoding,User-Agent', 'Content-Encoding': 'gzip', 'Date': 'Thu, 13 Jun 2019 22:43:31 GMT', 'Connection': 'keep-alive'} 

application/json; charset=UTF-8 
 

The following information pertains to the 10-year:3-month spread data
200 

{'Content-Type': 'application/json; charset=UTF-8', 'Content-Length': '15272', 'Server': 'Apache', 'Last-Modified': 'Thu, 13 Jun 2019 20:48:02 GMT', 'Expires': 'Thu, 13 Jun 2019 22:43:31 GMT', 'x-rate-limit-limit': '120', 'x-rate-limit-remaining': '119', 'Vary': 'Accept-Encoding,User-Agent', 'Content-Encoding': 'gzip', 'Date': 'Thu, 13 Jun 2019 22:43:31 GMT', 'Connection': 'keep-alive'} 

application/json; charset=UTF

Above, we see that the server returned a status code of 200.  This means that everything went as planned!  Additionally, we see that the data is in a JSON format with the UTF-8 character set.  We'll use this information next to decode the dataset so that it will be readable in Python.

In [2]:
#Create an empty dictionary to house the new data
data_2year = {}
data_3month = {}

#Assign the new data to the new dictionary and decode it (?)
data_2year = response_2year.content.decode("utf-8")
data_3month = response_3month.content.decode("utf-8")

#Decode the json data format so it's readable in Python (?)
data_2year = json.loads(data_2year) 
data_3month = json.loads(data_3month) 

print("The following information pertains to the 10-year:2-year spread data")
print(data_2year, '\n', '\n')
print("The following information pertains to the 10-year:3-month spread data")
print(data_3month)


The following information pertains to the 10-year:2-year spread data
{'realtime_start': '2019-06-13', 'realtime_end': '2019-06-13', 'observation_start': '2008-01-01', 'observation_end': '9999-12-31', 'units': 'lin', 'output_type': 1, 'file_type': 'json', 'order_by': 'observation_date', 'sort_order': 'desc', 'count': 2987, 'offset': 0, 'limit': 100000, 'observations': [{'realtime_start': '2019-06-13', 'realtime_end': '2019-06-13', 'date': '2019-06-12', 'value': '0.25'}, {'realtime_start': '2019-06-13', 'realtime_end': '2019-06-13', 'date': '2019-06-11', 'value': '0.22'}, {'realtime_start': '2019-06-13', 'realtime_end': '2019-06-13', 'date': '2019-06-10', 'value': '0.25'}, {'realtime_start': '2019-06-13', 'realtime_end': '2019-06-13', 'date': '2019-06-07', 'value': '0.24'}, {'realtime_start': '2019-06-13', 'realtime_end': '2019-06-13', 'date': '2019-06-06', 'value': '0.24'}, {'realtime_start': '2019-06-13', 'realtime_end': '2019-06-13', 'date': '2019-06-05', 'value': '0.29'}, {'realtime_

The 'observations' column has a whole lot more data in it than is displayed, and I think that's where we'll find the bond rates we're looking for <img src = 'https://starwarsblog.starwars.com/wp-content/uploads/2017/06/25-star-wars-quotes-obi-wan-kenobi-identification-tall.jpg'/>.

In [3]:
#Print only the sections of each dataset containing the observations
print("The following information pertains to the 10-year:2-year spread data")
df_2year = pd.DataFrame(data_2year['observations'])
print(df_2year.head())

print('\n', '\n', "The following information pertains to the 10-year:3-month spread data")
df_3month = pd.DataFrame(data_3month['observations'])
print(df_3month.head())

The following information pertains to the 10-year:2-year spread data
         date realtime_end realtime_start value
0  2019-06-12   2019-06-13     2019-06-13  0.25
1  2019-06-11   2019-06-13     2019-06-13  0.22
2  2019-06-10   2019-06-13     2019-06-13  0.25
3  2019-06-07   2019-06-13     2019-06-13  0.24
4  2019-06-06   2019-06-13     2019-06-13  0.24

 
 The following information pertains to the 10-year:3-month spread data
         date realtime_end realtime_start  value
0  2019-06-12   2019-06-13     2019-06-13  -0.11
1  2019-06-11   2019-06-13     2019-06-13  -0.12
2  2019-06-10   2019-06-13     2019-06-13  -0.14
3  2019-06-07   2019-06-13     2019-06-13  -0.19
4  2019-06-06   2019-06-13     2019-06-13  -0.21


We see that we have the following columns to work with:
- index: produced by the Pandas dataframe
- date: date the value was recorded
- realtime_end: the end date when the information was accurate before being changed
- realtime_start: the start date when the information was accurate before being changed
- value: the value of the 10-year less the 2-year (or 3-month) bond rate

While the realtime_end/start dates are useful in determining whether an update has occurred, we're really just concerned about the current state of the data. So, we'll keep just the 'date' column, and the 'value' column for each dataset at this time.  

I'll want to get a count of the number of missing values before I decide how to deal with that issue, so that'll be the first order of business on that point.

In [4]:
#Count the frequency of values in each dataframe
print(df_2year.groupby('value').size())
print(df_3month.groupby('value').size())

#Verify that the dates in the date columns are identical to confirm that values are properly aligned

#Change object types to datetime and numeric (float) values (needs to happen after NaNs are taken care of)
#print(final)
#final[['date_2year', 'date_3month']] = final[['date_2year', 'date_3month']].apply(pd.to_datetime) #convert the date column to a date datatype
#final[['value_2year', 'value_3month']] = final[['value_2year', 'value_3month']].apply(pd.to_numeric) #convert the value column to a float datatype


value
.       123
0.11      2
0.12      2
0.13      4
0.14      6
0.15      9
0.16     27
0.17     21
0.18     16
0.19     14
0.2       7
0.21     19
0.22      9
0.23      8
0.24     17
0.25     11
0.26     10
0.27     10
0.28     10
0.29     13
0.3       9
0.31      3
0.32      7
0.33      4
0.34      3
0.35      4
0.36      1
0.37      1
0.38      1
0.39      1
       ... 
2.60     17
2.61      9
2.62      9
2.63      5
2.64      4
2.65     11
2.66     11
2.67     14
2.68      8
2.69      7
2.70      9
2.71      9
2.72     15
2.73     11
2.74     14
2.75     13
2.76     14
2.77      8
2.78      6
2.79      5
2.80      5
2.81     18
2.82     22
2.83     11
2.84      7
2.85      6
2.87      4
2.89      3
2.90      2
2.91      1
Length: 281, dtype: int64
value
-0.01      1
-0.02      1
-0.03      2
-0.04      1
-0.05      3
-0.06      1
-0.11      2
-0.12      2
-0.14      1
-0.16      1
-0.19      1
-0.21      2
-0.23      2
-0.28      1
.        123
0.0        2
0.01       3
0.02     

In [14]:
#As it turns out, there are no NaN, null, or '0' values in the dataframe.  
#The blank values are currently held by periods ('.').  I'll replace these with NaN

#Change all values listed only as '.' with NaN (as Pandas counts NaN as a float, this should allow our calculations to work)
df_2year = df_2year.replace('.', np.nan)
df_3month = df_3month.replace('.', np.nan)

#Drop the 'realtime' columns from each dataset
# df_2year = df_2year.drop(columns = 'realtime_end')
# df_2year = df_2year.drop(columns = 'realtime_start')
# df_3month = df_3month.drop(columns = 'realtime_end')
# df_3month = df_3month.drop(columns = 'realtime_start')

#Print to confirm that '.' were swapped out for NaNs and that columns were dropped
print(df_2year, '\n', df_3month)

            value
date             
2019-06-12   0.25
2019-06-11   0.22
2019-06-10   0.25
2019-06-07   0.24
2019-06-06   0.24
2019-06-05   0.29
2019-06-04   0.24
2019-06-03   0.25
2019-05-31   0.19
2019-05-30   0.16
2019-05-29   0.16
2019-05-28   0.14
2019-05-27    NaN
2019-05-24   0.16
2019-05-23   0.19
2019-05-22   0.16
2019-05-21   0.17
2019-05-20   0.20
2019-05-17   0.19
2019-05-16   0.20
2019-05-15   0.21
2019-05-14   0.22
2019-05-13   0.22
2019-05-10   0.21
2019-05-09   0.19
2019-05-08   0.19
2019-05-07   0.17
2019-05-06   0.20
2019-05-03   0.21
2019-05-02   0.20
...           ...
2008-02-11   1.69
2008-02-08   1.71
2008-02-07   1.75
2008-02-06   1.65
2008-02-05   1.68
2008-02-04   1.60
2008-02-01   1.53
2008-01-31   1.50
2008-01-30   1.48
2008-01-29   1.40
2008-01-28   1.41
2008-01-25   1.38
2008-01-24   1.43
2008-01-23   1.42
2008-01-22   1.44
2008-01-21    NaN
2008-01-18   1.30
2008-01-17   1.22
2008-01-16   1.23
2008-01-15   1.19
2008-01-14   1.23
2008-01-11   1.23
2008-01-10

In [6]:
#Change the date columns to a datetime type
#df_2year['date']= df_2year.loc('date', axis=1).astype(dtype = 'datetime64[ns]')
#df_3month.date = df_3month.date.astype(dtype = 'datetime64[ns]')
#df_2year['date'] = pd.to_datetime(df_2year['date'])

#Set the date columns as the index
df_2year.set_index('date', inplace = True)
df_3month.set_index('date', inplace = True)

#Change value columns to float type so we can calculate an average for the missing values
df_2year.value = df_2year.value.astype(dtype = float)
df_3month.value = df_3month.value.astype(dtype = float)

print(df_2year, df_3month)

# print("The df_2year dataset: ", '\n',df_2year.head(n=15),'\n',)
# print("The df_3month dataset: ", '\n', df_3month.head(n=15))

#Printing to confirm that value column data types were changed
print("df_2year datatypes:", '\n', df_2year.dtypes, '\n')
print("df_3month datatypes:", '\n', df_3month.dtypes)


            value
date             
2019-06-12   0.25
2019-06-11   0.22
2019-06-10   0.25
2019-06-07   0.24
2019-06-06   0.24
2019-06-05   0.29
2019-06-04   0.24
2019-06-03   0.25
2019-05-31   0.19
2019-05-30   0.16
2019-05-29   0.16
2019-05-28   0.14
2019-05-27    NaN
2019-05-24   0.16
2019-05-23   0.19
2019-05-22   0.16
2019-05-21   0.17
2019-05-20   0.20
2019-05-17   0.19
2019-05-16   0.20
2019-05-15   0.21
2019-05-14   0.22
2019-05-13   0.22
2019-05-10   0.21
2019-05-09   0.19
2019-05-08   0.19
2019-05-07   0.17
2019-05-06   0.20
2019-05-03   0.21
2019-05-02   0.20
...           ...
2008-02-11   1.69
2008-02-08   1.71
2008-02-07   1.75
2008-02-06   1.65
2008-02-05   1.68
2008-02-04   1.60
2008-02-01   1.53
2008-01-31   1.50
2008-01-30   1.48
2008-01-29   1.40
2008-01-28   1.41
2008-01-25   1.38
2008-01-24   1.43
2008-01-23   1.42
2008-01-22   1.44
2008-01-21    NaN
2008-01-18   1.30
2008-01-17   1.22
2008-01-16   1.23
2008-01-15   1.19
2008-01-14   1.23
2008-01-11   1.23
2008-01-10

In [7]:
#For each row in the value columns, calculate the average of the value preceeding and following each NaN and replace the NaN with that new value
final_2year = (df_2year.value.ffill()+df_2year.value.bfill())/2
final_2year.value = df_2year.value.bfill().ffill()
final_3month = (df_2year.value.ffill()+df_2year.value.bfill())/2
final_3month.value = df_3month.value.bfill().ffill()

#For any NaN values at the beginning/end of the dataset, fill with the previous/following day's data

#Count to verify if there are 
nans_2year = sum(final_2year.value.isnull())
print("There are,", nans_2year, "NaN values in the df_2year dataset.")
nans_3month = sum(final_3month.value.isnull())
print("There are,", nans_3month, "NaN values in the df_3month dataset.")
print(final_2year)
print(final_3month)
###Two more options to calculate the average for the missing values and replace those currently listed as '.'
#final = final.fillna('value_2year', ffill, axis = 1).add(final.fillna('value_2year', bfill, axis = 1)).div(2)
#final.value_2year = final.value_2year.fillna((final.value_2year.shift() + final.value_2year.shift(-1))/2)

There are, 0 NaN values in the df_2year dataset.
There are, 0 NaN values in the df_3month dataset.
date
2019-06-12    0.25
2019-06-11    0.22
2019-06-10    0.25
2019-06-07    0.24
2019-06-06    0.24
2019-06-05    0.29
2019-06-04    0.24
2019-06-03    0.25
2019-05-31    0.19
2019-05-30    0.16
2019-05-29    0.16
2019-05-28    0.14
2019-05-27    0.15
2019-05-24    0.16
2019-05-23    0.19
2019-05-22    0.16
2019-05-21    0.17
2019-05-20    0.20
2019-05-17    0.19
2019-05-16    0.20
2019-05-15    0.21
2019-05-14    0.22
2019-05-13    0.22
2019-05-10    0.21
2019-05-09    0.19
2019-05-08    0.19
2019-05-07    0.17
2019-05-06    0.20
2019-05-03    0.21
2019-05-02    0.20
              ... 
2008-02-11    1.69
2008-02-08    1.71
2008-02-07    1.75
2008-02-06    1.65
2008-02-05    1.68
2008-02-04    1.60
2008-02-01    1.53
2008-01-31    1.50
2008-01-30    1.48
2008-01-29    1.40
2008-01-28    1.41
2008-01-25    1.38
2008-01-24    1.43
2008-01-23    1.42
2008-01-22    1.44
2008-01-21    1.37
200

In the print above, we see that there are no longer any NaN values

In [8]:
#Rename rename the values columns
final_2year.columns=['values_2year']
final_3month.columns=['values_3month']
# print(final_2year.head())
# print(final_2year.head())

# final_2year = final_2year.rename(index=str, columns={'date':'date_2year','value':'value_2year'})
# final_3month = final_3month.rename(index=str, columns={'date':'date_3month', 'value':'value_3month'})

#Join the two new dataframes into a single dataframe
final = pd.concat([final_2year, final_3month], axis=1, join_axes=[final_2year.index])

print(final)

            value  value
date                    
2019-06-12   0.25   0.25
2019-06-11   0.22   0.22
2019-06-10   0.25   0.25
2019-06-07   0.24   0.24
2019-06-06   0.24   0.24
2019-06-05   0.29   0.29
2019-06-04   0.24   0.24
2019-06-03   0.25   0.25
2019-05-31   0.19   0.19
2019-05-30   0.16   0.16
2019-05-29   0.16   0.16
2019-05-28   0.14   0.14
2019-05-27   0.15   0.15
2019-05-24   0.16   0.16
2019-05-23   0.19   0.19
2019-05-22   0.16   0.16
2019-05-21   0.17   0.17
2019-05-20   0.20   0.20
2019-05-17   0.19   0.19
2019-05-16   0.20   0.20
2019-05-15   0.21   0.21
2019-05-14   0.22   0.22
2019-05-13   0.22   0.22
2019-05-10   0.21   0.21
2019-05-09   0.19   0.19
2019-05-08   0.19   0.19
2019-05-07   0.17   0.17
2019-05-06   0.20   0.20
2019-05-03   0.21   0.21
2019-05-02   0.20   0.20
...           ...    ...
2008-02-11   1.69   1.69
2008-02-08   1.71   1.71
2008-02-07   1.75   1.75
2008-02-06   1.65   1.65
2008-02-05   1.68   1.68
2008-02-04   1.60   1.60
2008-02-01   1.53   1.53


In [9]:
import matplotlib.pyplot as plt
import matplotlib.dates as dates

plt.rcParams['figure.dpi'] = 200
ax = plt.axes()
#Generate the plot (figsize argument is in inches)


In [10]:
from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.plotting import figure

dates = np.array(final)
source = ColumnDataSource(data = dict(date = dates))

p=figure(plot_height = 300, plot_width=800, tools="xpan", toolbar_location=None, 
         x_axis_type="datetime", x_axis_location="above", background_fill_color="#efefef")
p.line('date', 'close', source = source)
p.yaxis.axis_label = 'Spread'

select = figure(title = 'Drag the middle and edges of teh selection box to change the range above', 
                plot_height = 130, plot_width = 800, y_range = p.y_range, x_axis_type = "datetime", y_axis_type = None,
                tools="", toolbar_location=None, background_fill_color="#efefef")

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.line('dates', 'value_2year', source = source)
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool

show(column(p, select))

ValueError: expected an instance of type Range1d, got DataRange1d(id='1003', ...) of type DataRange1d

In [ ]:
#How I did this at first:

#import urllib.request as request
    
#HTTPS GET Request (for JSON file type)
# with request.urlopen(url) as response:
#     source = response.read()
#     data = json.loads(source)
#     print(data)
    
# for categories in data:
#     print(categories)

[Maybe worth reading](https://www.federalreserve.gov/econres/notes/feds-notes/predicting-recession-probabilities-using-the-slope-of-the-yield-curve-20180301.htm)